In [14]:
import pandas as pd
import os 
from datetime import datetime

In [15]:
#Acesso Diretório dos envios feitos pelo Max
envio = 'C:\\Users\\SV - João Victor\\Sucesso em Vendas\\BI - Gestão de Resultados - Documentos\\EDMIL\\Edmil - 2022\\Dados\\'
arquivos = os.listdir(envio)

for arquivo in arquivos:
    if "Resultado Geral" in arquivo:
        envio_geral = pd.read_excel(envio + arquivo, sheet_name = 'Resultado Geral')
        envio_regional = pd.read_excel(envio + arquivo, sheet_name = 'Resultado Regionais')
        envio_filial = pd.read_excel(envio + arquivo, sheet_name = 'Resultado por Filial')
        envio_div = pd.read_excel(envio + arquivo, sheet_name = 'Resultado por Filial')
        envio_loja = pd.read_excel(envio + arquivo, sheet_name = 'Resultado por Filial')

In [16]:
#Acesso a Base de Dados
dir = "C:\\Users\\SV - João Victor\\Sucesso em Vendas\\BI - Gestão de Resultados - Documentos\\EDMIL\\Edmil - 2022\\base.xlsx"
base_venda = pd.read_excel(dir, sheet_name = " Base_Meta - Vendedor")
base_div = pd.read_excel(dir, sheet_name = "Base Divisão - 2022")
base_div_geral = pd.read_excel(dir, sheet_name = "Divisão por Lojas")
base_loja = pd.read_excel(dir, sheet_name = "Base Geral - Total")
base_regional = pd.read_excel(dir, sheet_name = "Base Regionais - 2022")
base_filial = pd.read_excel(dir, sheet_name = "Base Geral - Total")

In [17]:
#Obtenção do mês/ano atual 

currentMonth = str(datetime.now().month)
currentYear = str(datetime.now().year)
#data = "01" + "/0" + currentMonth + "/" + currentYear
data = "01/12/2021"

In [18]:
#ETL da Atualização Vendedores
head = envio_filial.iloc[0]
envio_filial = envio_filial[1:]
envio_filial.columns = head
envio_filial['Mês/Ano'] = data
totais = envio_filial.index[envio_filial['Divisão'].str.contains('Total', na = False)]
envio_filial.drop(totais, axis = 0, inplace = True)
envio_filial.drop(columns = ['Regional', 'Divisão'], inplace = True)
envio_filial["Chave"] = envio_filial["Atendente"] + " - " + envio_filial["Mês/Ano"]
envio_filial['Loja'] = envio_filial['Loja'].astype(str)
#envio_filial['Chaves'] = envio_filial['Mês/Ano'] + '-' + envio_filial['Atendente'] + '-' + envio_filial['Loja']
envio_filial = envio_filial.groupby(['Chave',"Mês/Ano" ,'Filial','Loja','Atendente'], as_index = False).sum()
#envio_filial.drop(envio_filial['Filial'],axis = 1, inplace = True)
envio_filial.rename(columns= {'Mês/Ano' : 'Data'}, inplace = True)


In [19]:
#ETL da Atualização Divisão
head = envio_geral.iloc[0]
envio_geral = envio_geral[2:]
envio_geral.columns = head
envio_geral['Mês/Ano'] = data

In [20]:
#ETL da Atualização de Divisão GERAL! 
head = envio_div.iloc[0]
envio_div = envio_div[1:-1]
envio_div.columns = head

totais = envio_div.index[envio_div['Divisão'].str.contains('Total', na = False)]
envio_div.drop(totais, axis = 0, inplace = True)
envio_div[['Número','Lojas']] = envio_div['Filial'].str.split(" - ", expand = True)
envio_div.drop(columns = ["Loja",'Filial'], inplace = True)
envio_div["Mês/Ano"] = data

In [21]:
#ETL da Atualização da envio Regionais 
head = envio_regional.iloc[0]
envio_regional = envio_regional[1:]
envio_regional.columns = head
arroz = envio_regional.index[envio_regional['Divisão'].str.contains("Total", na = False)]
envio_regional.drop(arroz, axis = 0, inplace = True)
envio_regional['Mês/Ano'] = data

In [22]:
#ETL da Atualização do envio Loja
head = envio_loja.iloc[0]
envio_loja = envio_loja[1:-1]
envio_loja.columns = head

totais = envio_loja.index[envio_loja['Divisão'].str.contains('Total', na = False)]
envio_loja.drop(totais, axis = 0, inplace = True)

envio_loja[['Número','Nome']] = envio_loja['Filial'].str.split(" - ", expand = True)
columns_to_drop = ['Número', 'Filial','Loja', 'Regional']
envio_loja.drop(columns_to_drop,axis = 1, inplace = True)
envio_loja['Mês/Ano'] = data
envio_loja.rename(columns= {"Mês/Ano" : "Data", "Nome" : "Loja"}, inplace= True)

In [23]:
#Concatenação das Bases com os Envios
venda = [base_venda, envio_filial]
base_vendedor = pd.concat(venda).reset_index(drop = True)

div = [base_div, envio_geral]
base_geral = pd.concat(div).reset_index(drop = True)

div_loja = [base_div_geral, envio_div]
base_div_loja = pd.concat(div_loja).reset_index(drop = True)

reg = [base_regional, envio_regional]
base_regional = pd.concat(reg).reset_index(drop = True)

loja = [base_loja, envio_loja]
base_meta = pd.concat(loja).reset_index(drop = True)

In [24]:
destino = 'C:\\Users\\SV - João Victor\\Sucesso em Vendas\\BI - Gestão de Resultados - Documentos\\EDMIL\\Edmil - 2022\\'
with pd.ExcelWriter(path = destino + "base.xlsx", engine = 'xlsxwriter', engine_kwargs = {'options' : {'strings_to_formulas': False}}) as update:
    base_regional = base_regional.to_excel(update, sheet_name = "Base Regionais - 2022")
    base_filial = base_filial.to_excel(update, sheet_name = "Base_Meta - Lojas")
    base_geral = base_geral.to_excel(update, sheet_name = "Base Divisão - 2022")
    base_meta = base_meta.to_excel(update, sheet_name = "Base Geral - Total")
    base_vendedor = base_vendedor.to_excel(update, sheet_name = " Base_Meta - Vendedor")
    base_div_loja = base_div_loja.to_excel(update, sheet_name = 'Divisão por Lojas')
